In [1]:
from dask.distributed import Client
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [2]:
c = Client('tcp://localhost:8786')

/home/appuser/.local/lib/python3.8/site-packages/distributed/client.py:1148: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| lz4     | None          | 3.1.1         | 3.1.1         |
| msgpack | 1.0.2         | 1.0.0         | 1.0.0         |
| numpy   | 1.20.3        | 1.18.1        | 1.18.1        |
| python  | 3.8.5.final.0 | 3.8.0.final.0 | 3.8.0.final.0 |
+---------+---------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [3]:
c.restart()

Client Scheduler: tcp://localhost:8786 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 2 Memory: 3.84 GiB


In [7]:
with open('repos.json') as json_file:
    repos = json.load(json_file)

In [8]:
df = pd.DataFrame(repos.values())

In [10]:
df = df.drop(['owner', 'name', 'id', 'full_name', 'created_at', 'last_commit', 'main_language'], axis=1)
X_pd = df.drop(['stargazer_count'], axis=1)
y_pd = df['stargazer_count']

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pd, y_pd, test_size=0.3, random_state=0)

In [12]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()

In [13]:
# HYPERTUNING
grid = dict(pca__n_components=[5, 10, 15, 20],
    forest__n_estimators=[50, 100, 150, 200],
            forest__max_depth=[5, 10, 100, 150],
            forest__ccp_alpha=[0.001, 0.01, 0.1, 0],
            )

In [14]:
from sklearn import linear_model, decomposition
from sklearn.pipeline import Pipeline

forest = RandomForestRegressor()
pca = decomposition.PCA()

pipe = Pipeline(steps=[('pca', pca),
                       ('forest', forest)])

In [15]:
grid_search = GridSearchCV(pipe,
                           param_grid=grid,
                           n_jobs=-1)

In [16]:
%%time
import blosc
from sklearn.datasets import make_classification

with joblib.parallel_backend('dask', scatter=[X_train, y_train]):
    grid_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [40]:
grid_search.score(X_test, y_test)

0.5339673411707899